<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Tutorial on generating an explanation for a text-based model on Watson OpenScale

This notebook includes steps for creating a text-based watson-machine-learning model, creating a subscription, configuring explainability, and finally generating an explanation for a transaction.

### Contents
- [1. Setup](#setup)
- [2. Creating and deploying a text-based model](#deploy)
- [3. Subscriptions](#subscription)
- [4. Explainability](#explainability)

**Note**: If using Watson Studio, try running the notebook on atleast 'Default Python 3.5 XS' version for faster results.

<a id="setup"></a>
## 1. Setup

### 1.1 Install Watson OpenScale and WML packages

In [1]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [2]:
!pip install --upgrade ibm-watson-machine-learning --no-cache | tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [3]:
CLOUD_API_KEY = "wqMwzhQ7GRNBlH5iS3OVGlf3Ym4OD4EPJjaA6Eg8mQgg"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [4]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
}

## 2. Creating and deploying a text-based model <a id="deploy"></a>

The dataset used is the UCI-ML SMS Spam Collection Dataset which can be found here: https://archive.ics.uci.edu/ml/machine-learning-databases/00228/. It is a binary classification dataset with the labels being 'ham' and 'spam'.

### 2.1 Loading the training data

In [ ]:
!rm -rf SMSSpam.csv
!wget 'https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/spam_detection/SMSSpam.csv'

In [6]:
# The training data is downloaded and saved as 'SMSSpam.csv' in this step from public link

# !pip install pandas
# !rm smsspamcollection.zip
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
# !unzip smsspamcollection.zip
#pd.read_csv("smsspamcollection.zip",sep="\t",header=None, encoding="utf-8").to_csv("SMSSpam.csv", header=["label", "text"], sep=",", index=False)

# !rm SMSSpamCollection
# !rm readme
# !rm smsspamcollection.zip

### 2.2 Creating a model

**Note**: Skip the pyspark install step below if you are using a Spark kernel on Watson Studio.

In [7]:
!pip install pyspark==2.4.0

     |████████████████████████████████| 213.4 MB 18 kB/s s eta 0:00:01/s eta 0:00:26��▏               | 108.0 MB 48.2 MB/s eta 0:00:03��█▊            | 131.5 MB 61.4 MB/s eta 0:00:02     |███████████████████████▌        | 156.9 MB 61.4 MB/s eta 0:00:01MB/s eta 0:00:03��█████████████▋    | 184.1 MB 20.0 MB/s eta 0:00:02��████████████████████████▊   | 191.7 MB 20.0 MB/s eta 0:00:02ta 0:00:01
     |████████████████████████████████| 197 kB 57.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.0-py2.py3-none-any.whl size=213793601 sha256=96433cc038b4253e74f7a01c9280b0e756a185e8df49987a60892d88f6da485d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/16/f1/95/5f30ebf85b300509e4dbce37d94daf7af58269db2e3af17b47
Successfully built pyspark


**Note**: When running this notebook locally, If the `SparkSession` import fails below, set 'SPARK_HOME' environment variable with the path to `pyspark` installation.

In [5]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv(path="SMSSpam.csv", header=True, multiLine=True, escape='"')
df.show(5, truncate = False)

Exception: Java gateway process exited before sending its port number

In [9]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=12345)
print("Total count of data set: {}".format(df.count()))
print("Total count of training data set: {}".format(train_df.count()))
print("Total count of test data set: {}".format(test_df.count()))

Total count of data set: 5572
Total count of training data set: 4420
Total count of test data set: 1152


In [10]:
!pip install nltk
from pyspark.ml.feature import StringIndexer, IndexToString, CountVectorizer, Tokenizer, IDF, StopWordsRemover
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

stringIndexer_label = StringIndexer(inputCol="label", outputCol="label_ix").fit(df)
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopword_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words").setStopWords(stop_words)
count = CountVectorizer(inputCol="filtered_words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
nb = GBTClassifier(labelCol="label_ix")
labelConverter = IndexToString(inputCol="prediction", outputCol="predictionLabel", labels=stringIndexer_label.labels)

[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
pipeline = Pipeline(stages=[stringIndexer_label, tokenizer, stopword_remover, count, idf, nb, labelConverter])
model = pipeline.fit(train_df)
predictions = model.transform(test_df)
evaluator = BinaryClassificationEvaluator(labelCol="label_ix", rawPredictionCol="prediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print("Area under ROC curve = %g" % auc)

Area under ROC curve = 0.846312


In [12]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.34'

2020-11-04 20:08:49,107 - ibm_watson_machine_learning.wml_client_error - WARNING - Missing meta_prop with name: 'type'.
2020-11-04 20:48:05,449 - ibm_watson_machine_learning.wml_client_error - WARNING - Scoring data input 'ScoringMetaNames.INPUT_DATA' is mandatory for synchronous scoring


In [15]:
wml_client.spaces.list(limit=10)

------------------------------------  --------------  ------------------------
ID                                    NAME            CREATED
7fb100d2-bfb3-40f6-8a61-9daf733a16aa  prod-space      2020-10-26T22:28:31.105Z
4892166c-6285-47f8-9f3f-b9c3c541a4a2  pre-prod-space  2020-10-26T21:25:32.230Z
a2c5ddbd-c002-4c86-ab07-ce7334fbfa4d  test_v4         2020-09-30T19:38:26.229Z
------------------------------------  --------------  ------------------------


In [16]:
WML_SPACE_ID='a2c5ddbd-c002-4c86-ab07-ce7334fbfa4d' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [13]:
MODEL_NAME = "Text Binary Classifier"

In [17]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_2.4")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.SPACE_UID: WML_SPACE_ID,
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_2.4",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        #wml_client._models.ConfigurationMetaNames.TRAINING_DATA_REFERENCES: training_data_references,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "label",
    }

Software Specification ID: 390d21f8-e58b-4fac-9c55-d7ceda621326


In [18]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_data=train_df, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: 6453464c-f02e-4feb-b700-c364e3e3a16a


In [19]:
!rm SMSSpam.csv

### 2.3 Deploying the model

In [21]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(MODEL_NAME + " deployment"),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: '6453464c-f02e-4feb-b700-c364e3e3a16a' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8169dcac-3068-4dc9-84a7-05f18fd42aec'
------------------------------------------------------------------------------------------------


Scoring URL:https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8169dcac-3068-4dc9-84a7-05f18fd42aec/predictions
Model id: 6453464c-f02e-4feb-b700-c364e3e3a16a
Deployment id: 8169dcac-3068-4dc9-84a7-05f18fd42aec


## 3. Subscriptions <a id="subscription"></a>

### 3.1 Configuring OS

In [22]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment this line if using IAM token to authenticate
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.1'

**Note**: Please re-run the above cell if it doesn't work the first time.

In [23]:
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True

In [45]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56


In [25]:
SERVICE_PROVIDER_NAME = "Watson Machine Learning V2_test"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [26]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [27]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN to initiate client
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 457d36c4-6fa3-4300-be21-785efb13fe53 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [31]:
asset_deployment_details_list = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id = WML_SPACE_ID).result['resources']
DEPLOYMENT_NAME='Text Binary Classifier deployment' # use the model name here 
asset_deployment_details = [asset for asset in asset_deployment_details_list if asset['entity']["name"]==DEPLOYMENT_NAME]

if len(asset_deployment_details)>0:
    [asset_deployment_details] = asset_deployment_details
else:
    raise ValueError('deployment with name "{}" not found.'.format(DEPLOYMENT_NAME))
asset_deployment_details

{'metadata': {'guid': '8169dcac-3068-4dc9-84a7-05f18fd42aec',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8169dcac-3068-4dc9-84a7-05f18fd42aec?space_id=a2c5ddbd-c002-4c86-ab07-ce7334fbfa4d',
  'created_at': '2020-11-04T20:12:57.059Z',
  'modified_at': '2020-11-04T20:12:57.059Z'},
 'entity': {'name': 'Text Binary Classifier deployment',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8169dcac-3068-4dc9-84a7-05f18fd42aec/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [32]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)
model_asset_details_from_deployment

{'metadata': {'guid': '8169dcac-3068-4dc9-84a7-05f18fd42aec',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8169dcac-3068-4dc9-84a7-05f18fd42aec?space_id=a2c5ddbd-c002-4c86-ab07-ce7334fbfa4d',
  'created_at': '2020-11-04T20:12:57.059Z',
  'modified_at': '2020-11-04T20:12:57.059Z'},
 'entity': {'name': 'Text Binary Classifier deployment',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/8169dcac-3068-4dc9-84a7-05f18fd42aec/predictions'},
  'asset': {'asset_id': '6453464c-f02e-4feb-b700-c364e3e3a16a',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/6453464c-f02e-4feb-b700-c364e3e3a16a?space_id=a2c5ddbd-c002-4c86-ab07-ce7334fbfa4d&version=2020-06-12',
   'name': 'Text Binary Classifier',
   'asset_type': 'model',
   'created_at': '2020-11-04T20:11:47.744Z',
   'modified_at': '2020-11-04T20:11:54.965Z'},
  'asset_properties': {'model_type': 'mllib_2.4',
   'runtime_environment': 'spark-2.4',
   'label_column': 

### 3.2 Subscribe the asset

In [34]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', model_uid)

Deleted existing subscription for model 6453464c-f02e-4feb-b700-c364e3e3a16a


In [35]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.UNSTRUCTURED_TEXT,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['metadata']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='label',
            probability_fields=['probability'],
            prediction_field='predictionLabel',
            feature_fields = ["text"],
            categorical_fields = ["text"],
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

'0c9f0711-d184-4a98-84df-ef169e8f0012'

In [39]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  e68217cf-1839-4cb6-95a1-c5a32ddc9dc1


### 3.3 Get subscription

In [ ]:
aios_client.data_mart.subscriptions.list()

In [ ]:
subscription.get_details()

### 3.4 Score the model and get transaction-id

In [37]:
text = "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"
payload = {"input_data": [{"fields": ["text"], "values": [[text]]}]}

response = wml_client.deployments.score(deployment_uid,payload)
print(response)

In [40]:
wos_client.data_sets.get_records_count(payload_data_set_id)

1

## 4. Explainability

### 4.1 Configure Explainability

In [41]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation da42c8b2-4ca0-43c2-bacd-163f5636b033 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### 4.2 Get explanation for the transaction

In [42]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)

Running explanations on scoring IDs: ['6729ea5feebf7ce567cf635df2017a03-1']
{
  "metadata": {
    "explanation_task_ids": [
      "1566ad35-5bee-4412-8a0a-94241112301a"
    ],
    "created_by": "IBMid-310002F0G1",
    "created_at": "2020-11-04T20:52:17.962841Z"
  }
}


In [44]:
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

{'metadata': {'explanation_task_id': '1566ad35-5bee-4412-8a0a-94241112301a',
  'created_by': 'IBMid-310002F0G1',
  'created_at': '2020-11-04T20:52:17.962841Z',
  'updated_at': '2020-11-04T20:52:32.942615Z'},
 'entity': {'status': {'state': 'finished'},
  'asset': {'id': '6453464c-f02e-4feb-b700-c364e3e3a16a',
   'name': 'Text Binary Classifier',
   'input_data_type': 'unstructured_text',
   'problem_type': 'binary',
   'deployment': {'id': '8169dcac-3068-4dc9-84a7-05f18fd42aec',
    'name': 'Text Binary Classifier deployment'}},
  'input_features': [{'name': 'text',
    'value': 'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info'}],
  'perturbed': False,
  'explanations': [{'explanation_type': 'lime',
    'predictions': [{'explanation_features': [{'feature_value': 'win',
        'weight': 0.39503435397824244,
        'positions': [[15, 18]]},
       {'feature_value': 'chances',
        'weight': 0.07